In [ ]:
import scipy.io as sio

# Load the .mat file
mat_contents = sio.loadmat('/content/My drive/SynthText/SynthTextgt.mat')

# Assuming 'imnames', 'wordBB', 'charBB', 'txt' are the keys
imnames = mat_contents['imnames']
wordBB = mat_contents['wordBB']
charBB = mat_contents['charBB']
txt = mat_contents['txt']

# Select indices of the images you want to keep
# Example: Keep first 1000 images of each selected folder
indices_to_keep = range(1000)  # adjust this based on your criteria

# Update the dictionaries by retaining only selected indices
mat_contents['imnames'] = imnames[:, indices_to_keep]
mat_contents['wordBB'] = wordBB[:, indices_to_keep]
mat_contents['charBB'] = charBB[:, indices_to_keep]
mat_contents['txt'] = txt[:, indices_to_keep]

# Save the modified .mat file
sio.savemat('modified_gt.mat', mat_contents)


Removing


In [ ]:
import os
import random
import shutil
import scipy.io as sio

# Path to the SynthText directory and gt.mat file
synthtext_dir = '/content/My drive/SynthText/'
gt_file_path = '/content/My drive/SynthText/SynthTextgt.mat'

# List all directories in the SynthText folder
all_folders = [f for f in os.listdir(synthtext_dir) if os.path.isdir(os.path.join(synthtext_dir, f))]

# Randomly select 20 folders
selected_folders = random.sample(all_folders, 20)

# Remove non-selected folders
for folder in all_folders:
    if folder not in selected_folders:
        shutil.rmtree(os.path.join(synthtext_dir, folder))

# Load the original gt.mat file
mat_contents = sio.loadmat(gt_file_path)

# Filter the annotations to include only those in the selected folders
def filter_annotations(data, selected_folders):
    filtered_indices = [i for i, name in enumerate(data['imnames'][0]) if any(folder in name[0] for folder in selected_folders)]
    return data[:, filtered_indices]

# Update the .mat content
mat_contents['imnames'] = filter_annotations(mat_contents['imnames'], selected_folders)
mat_contents['wordBB'] = filter_annotations(mat_contents['wordBB'], selected_folders)
mat_contents['charBB'] = filter_annotations(mat_contents['charBB'], selected_folders)
mat_contents['txt'] = filter_annotations(mat_contents['txt'], selected_folders)

# Save the updated .mat file
sio.savemat('updated_gt.mat', mat_contents)


In [ ]:
import os
import scipy.io as sio


# Load the .mat file
mat_contents = sio.loadmat(gt_file_path, squeeze_me=True)

# Extract list of image filenames to keep from the .mat file
images_to_keep = set([img[0] for img in mat_contents['imnames'][0]])

# Function to delete images not listed in the gt.mat file
def delete_unlisted_images(root_dir, keep_set):
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".jpg"):
                # Check if the image is in the set of images to keep
                image_path = os.path.relpath(os.path.join(subdir, file), start=root_dir)
                if image_path not in keep_set:
                    os.remove(os.path.join(subdir, file))
                    print(f"Deleted: {os.path.join(subdir, file)}")

# Call the function to delete unlisted images
delete_unlisted_images(synthtext_dir, images_to_keep)
